In [46]:
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForCausalLM
import string

In [ ]:
## 外部のデータセットをインポート

rs = "izumi-lab/llm-japanese-dataset"
datasets = load_dataset(rs)
split_ix = round(len(datasets['train']) * 0.7)
datasets = DatasetDict({
    "train": datasets['train'][:split_ix],
    "val": datasets['train'][split_ix:]
})
datasets

In [5]:
## モデル名

model_name = "cyberagent/calm2-7b-chat"

In [6]:
## トークナイザーのインポート

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    use_fast = True
)

In [7]:
## モデルのインポート
## リソースが足りないので、1/4量子化

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map = 'auto', # 複数のGPUがあるときに、それらを均等に使用する
    load_in_8bit = True, # 精度と引き換えに、パラメータを8bitに省略したLLMを使用して計算を軽量化
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.04G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [30]:
chat_template = string.Template(
    "USER:${user}\nASSISTANT:"
)

In [35]:
sample_text = chat_template.safe_substitute({"user": "人気タレント・タモリの本名は何でしょう？"})

In [36]:
def generate(model, text):
    input_ids = tokenizer.encode(
        text,
        return_tensors = 'pt',
        add_special_tokens = True
    ).to(model.device)
    output_ids = model.generate(
        input_ids,
        max_new_tokens = 100,
        do_sample = True,
        temperature = 0.8,
    )
    print(
        tokenizer.decode(
            output_ids[0],
            skip_special_tokens = True
        )
    )

In [37]:
%%time
generate(model, sample_text)

USER:人気タレント・タモリの本名は何でしょう？
ASSISTANT: タモリの本名は「森田一義」です。
CPU times: user 2.19 s, sys: 8.41 ms, total: 2.2 s
Wall time: 2.2 s
